# 02 - Genetic Algorithm Implementation

This notebook implements and tests a Genetic Algorithm to solve the Tourist Trip Design Problem.

## Import Libraries

In [ ]:
import sys
sys.path.append('../scripts')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from data_utils import load_attractions_data, calculate_distance_matrix
from ga_core import GeneticAlgorithm
from visualization import (
    plot_fitness_evolution,
    plot_route_on_map,
    plot_tour_statistics,
    create_summary_report
)

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Load Data

In [ ]:
# Load attractions data
attractions = load_attractions_data('../data/sri_lanka_attractions.csv')
print(f"Loaded {len(attractions)} attractions")

# Calculate or load distance matrix
try:
    distance_matrix = np.load('../data/distance_matrix.npy')
    print("Distance matrix loaded from file")
except:
    distance_matrix = calculate_distance_matrix(attractions)
    np.save('../data/distance_matrix.npy', distance_matrix)
    print("Distance matrix calculated and saved")

# Extract data for optimization
scores = attractions['score'].values
visit_durations = attractions['visit_duration'].values

print(f"\nData loaded successfully!")
attractions.head()

## Define Problem Parameters

In [ ]:
# Define problem constraints
MAX_TIME = 24  # Maximum time available for the trip (hours)

# Genetic Algorithm parameters
POPULATION_SIZE = 100
GENERATIONS = 500
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8

print("Problem Parameters:")
print(f"Maximum trip time: {MAX_TIME} hours")
print(f"\nGA Parameters:")
print(f"Population size: {POPULATION_SIZE}")
print(f"Generations: {GENERATIONS}")
print(f"Mutation rate: {MUTATION_RATE}")
print(f"Crossover rate: {CROSSOVER_RATE}")

## Initialize and Run Genetic Algorithm

In [ ]:
# Initialize GA
ga = GeneticAlgorithm(
    distance_matrix=distance_matrix,
    scores=scores,
    visit_durations=visit_durations,
    max_time=MAX_TIME,
    population_size=POPULATION_SIZE,
    generations=GENERATIONS,
    mutation_rate=MUTATION_RATE,
    crossover_rate=CROSSOVER_RATE
)

print("Genetic Algorithm initialized!")
print("\nRunning optimization...")

# Run the algorithm
start_time = time.time()
best_solution, best_fitness, fitness_history = ga.evolve()
end_time = time.time()

computation_time = end_time - start_time

print(f"\nOptimization completed in {computation_time:.2f} seconds")
print(f"Best fitness achieved: {best_fitness:.2f}")

## Extract Valid Tour

In [ ]:
# Get valid tour within time constraint
valid_tour = ga.get_valid_tour(best_solution)

print(f"Valid tour contains {len(valid_tour)} attractions")
print("\nTour sequence:")
for i, idx in enumerate(valid_tour):
    print(f"{i+1}. {attractions.iloc[idx]['name']} (Score: {attractions.iloc[idx]['score']}, Duration: {attractions.iloc[idx]['visit_duration']}h)")

## Visualize Fitness Evolution

In [ ]:
# Plot fitness evolution
fig = plot_fitness_evolution(fitness_history)
plt.show()

## Analyze Convergence

In [ ]:
# Analyze convergence speed
max_fitnesses = [h['max_fitness'] for h in fitness_history]
first_95_percent = None

final_fitness = max_fitnesses[-1]
threshold = 0.95 * final_fitness

for i, fitness in enumerate(max_fitnesses):
    if fitness >= threshold:
        first_95_percent = i
        break

if first_95_percent:
    print(f"Algorithm reached 95% of final fitness at generation {first_95_percent}")
    print(f"Convergence rate: {(first_95_percent/GENERATIONS)*100:.1f}% of total generations")
else:
    print("Algorithm did not reach 95% of final fitness")

## Visualize Tour Statistics

In [ ]:
# Plot comprehensive tour statistics
fig = plot_tour_statistics(attractions, valid_tour, distance_matrix)
plt.show()

## Create Interactive Map

In [ ]:
# Create interactive map of the tour
tour_map = plot_route_on_map(attractions, valid_tour, output_file='../data/ga_tour_map.html')
print("Interactive map created! Open ../data/ga_tour_map.html in a browser to view.")

## Generate Summary Report

In [ ]:
# Generate and print summary report
report = create_summary_report(attractions, valid_tour, distance_matrix, algorithm_name='Genetic Algorithm')
print(report)

# Save report to file
with open('../data/ga_tour_report.txt', 'w') as f:
    f.write(report)
print("Report saved to ../data/ga_tour_report.txt")

## Sensitivity Analysis

In [ ]:
# Test different time constraints
time_constraints = [12, 18, 24, 30, 36]
results = []

print("Running sensitivity analysis for different time constraints...")

for max_time in time_constraints:
    ga_test = GeneticAlgorithm(
        distance_matrix=distance_matrix,
        scores=scores,
        visit_durations=visit_durations,
        max_time=max_time,
        population_size=50,
        generations=200,
        mutation_rate=0.1,
        crossover_rate=0.8
    )
    
    solution, fitness, _ = ga_test.evolve()
    tour = ga_test.get_valid_tour(solution)
    
    results.append({
        'max_time': max_time,
        'attractions_visited': len(tour),
        'total_score': fitness
    })
    
    print(f"  Max time: {max_time}h → {len(tour)} attractions, Score: {fitness:.2f}")

# Plot sensitivity analysis
results_df = pd.DataFrame(results)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(results_df['max_time'], results_df['attractions_visited'], marker='o', linewidth=2, markersize=8)
ax1.set_xlabel('Maximum Trip Time (hours)')
ax1.set_ylabel('Number of Attractions Visited')
ax1.set_title('Attractions vs Time Constraint')
ax1.grid(True, alpha=0.3)

ax2.plot(results_df['max_time'], results_df['total_score'], marker='s', color='orange', linewidth=2, markersize=8)
ax2.set_xlabel('Maximum Trip Time (hours)')
ax2.set_ylabel('Total Satisfaction Score')
ax2.set_title('Total Score vs Time Constraint')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Save Results

In [ ]:
# Save GA results
results_data = {
    'algorithm': 'Genetic Algorithm',
    'best_solution': best_solution,
    'valid_tour': valid_tour,
    'best_fitness': best_fitness,
    'computation_time': computation_time,
    'fitness_history': fitness_history,
    'parameters': {
        'population_size': POPULATION_SIZE,
        'generations': GENERATIONS,
        'mutation_rate': MUTATION_RATE,
        'crossover_rate': CROSSOVER_RATE,
        'max_time': MAX_TIME
    }
}

np.save('../data/ga_results.npy', results_data)
print("Results saved to ../data/ga_results.npy")

## Next Steps

In the next notebook, we will implement a Mixed Integer Programming (MIP) model as a benchmark to compare with the Genetic Algorithm results.